# LSTM Test - Kaza - No PCA

In [ ]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src.config import PROJECT_ROOT
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader

In [ ]:
df = pd.read_parquet(PROJECT_ROOT.joinpath('data/interim/sampleKaza.parquet'))
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [ ]:
# PCA kullanılmıyor
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
predictor = TrafficPredictor('lstm', dataset.sensors, X_STEP, Y_STEP, pca_model=dataset.pca if False else None)
predictor.load_model(PROJECT_ROOT.joinpath('models/lstm_kaza_nopca.pt'))
loader = DataLoader(dataset, batch_size=10)
preds = predictor.inference(loader)
preds[:5]